Defaulting to user installation because normal site-packages is not writeable
  Using cached zipfile36-0.1.3-py3-none-any.whl.metadata (736 bytes)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [8]:
import requests
import zipfile
import io
import pandas as pd
from datetime import date

# Set the date for the file you want to download
# Using a specific date that is known to be available
# Change this date as needed, for example: date_to_download = date(2025, 9, 21)
date_to_download = date(2025, 9, 21)
date_str = date_to_download.strftime("%Y%m%d")

# Construct the URL
url = f"http://data.gdeltproject.org/events/{date_str}.export.CSV.zip"
print(f"Attempting to download GDELT file for {date_str}...")

try:
    # Download the compressed file
    response = requests.get(url)
    
    # Check if the download was successful
    if response.status_code == 200:
        print("File downloaded successfully.")
        
        # Read the zip content into memory
        zip_content = io.BytesIO(response.content)
        
        # Decompress the CSV file and load into a pandas DataFrame
        with zipfile.ZipFile(zip_content, 'r') as zip_ref:
            csv_file_name = zip_ref.namelist()[0]
            df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')
        
        print("Data loaded into DataFrame.")
        
        # Define the Excel file name
        excel_filename = f"gdelt_events_{date_str}.xlsx"
        
        # Export the DataFrame to an Excel file
        df.to_excel(excel_filename, index=False)
        
        print(f"\nData successfully exported to '{excel_filename}'.")
    elif response.status_code == 404:
        print(f"Error 404: The file for {date_str} was not found on the GDELT server.")
        print("Possible reasons: The data has not been published yet or the date is incorrect.")
        print("Please check the GDELT data download page to confirm availability.")
    else:
        print(f"An unexpected error occurred with status code: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Network error: {e}")
except zipfile.BadZipFile:
    print("The downloaded file is not a valid ZIP file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Attempting to download GDELT file for 20250921...
File downloaded successfully.


C:\Users\jeanc\AppData\Local\Temp\ipykernel_38960\2090589138.py:31: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')


Data loaded into DataFrame.

Data successfully exported to 'gdelt_events_20250921.xlsx'.


In [17]:
import requests
import zipfile
import io
import pandas as pd
from datetime import date, timedelta

# =========================================================================
# === PARTIE 1 : Définition des intervalles de dates que vous pouvez modifier ===
# =========================================================================

# Option 1 : Période dynamique (basée sur la date d'aujourd'hui)
# yesterday = date.today() - timedelta(days=1)
# start_date = yesterday - timedelta(days=6)  # 6 jours avant hier = 7 jours au total

# Option 2 : Période fixe (pour analyser une plage spécifique)
# Décommentez les deux lignes ci-dessous si vous voulez une période fixe
start_date = date(2025, 9, 10)  # Changez l'année, le mois et le jour
yesterday = date(2025, 9, 22)    # Changez l'année, le mois et le jour

# =========================================================================
# === PARTIE 2 : Boucle de téléchargement et de traitement des données ===
# =========================================================================

# Liste pour stocker les DataFrames de chaque jour
all_data = []

# Calcul du nombre de jours dans l'intervalle
num_days = (yesterday - start_date).days + 1
print(f"Téléchargement des données GDELT du {start_date} au {yesterday} ({num_days} jours)...")

# Boucle sur la période définie
for i in range(num_days):
    current_date = start_date + timedelta(days=i)
    date_str = current_date.strftime("%Y%m%d")
    url = f"http://data.gdeltproject.org/events/{date_str}.export.CSV.zip"

    try:
        response = requests.get(url)

        if response.status_code == 200:
            print(f"✅ Fichier pour le {date_str} téléchargé avec succès.")
            zip_content = io.BytesIO(response.content)

            with zipfile.ZipFile(zip_content, 'r') as zip_ref:
                csv_file_name = zip_ref.namelist()[0]
                # Le format GDELT utilise la tabulation comme séparateur
                df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')
            
            df['Date'] = current_date
            all_data.append(df)
            
        elif response.status_code == 404:
            print(f"❌ Le fichier pour le {date_str} n'a pas été trouvé (Erreur 404).")
        else:
            print(f"⚠️ Erreur inattendue pour le {date_str} : {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"⚠️ Erreur de réseau lors du téléchargement pour le {date_str} : {e}")
    except Exception as e:
        print(f"⚠️ Une erreur s'est produite lors du traitement du fichier pour le {date_str} : {e}")

# =========================================================================
# === PARTIE 3 : Consolidation et affichage des résultats ===
# =========================================================================

# Consolidation des données si des fichiers ont été téléchargés
if all_data:
    print("\n📦 Consolidation de tous les DataFrames en un seul...")
    final_df = pd.concat(all_data, ignore_index=True)
    
    print(f"\n✅ Données chargées dans le DataFrame 'final_df'.")
    print(f"Taille du DataFrame : {final_df.shape[0]} lignes et {final_df.shape[1]} colonnes.")
    
    # Affiche les 5 premières lignes du DataFrame pour vérification
    print("\nAperçu du DataFrame :")
    print(final_df.head())
else:
    final_df = pd.DataFrame()
    print("\n❌ Aucune donnée n'a été téléchargée. Le DataFrame 'final_df' est vide.")

# La variable 'final_df' contient maintenant toutes les données.
# Vous pouvez continuer à travailler sur ce DataFrame directement.

Téléchargement des données GDELT du 2025-09-10 au 2025-09-22 (13 jours)...
✅ Fichier pour le 20250910 téléchargé avec succès.
✅ Fichier pour le 20250911 téléchargé avec succès.
✅ Fichier pour le 20250912 téléchargé avec succès.


C:\Users\jeanc\AppData\Local\Temp\ipykernel_38960\424091.py:47: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')


✅ Fichier pour le 20250913 téléchargé avec succès.


C:\Users\jeanc\AppData\Local\Temp\ipykernel_38960\424091.py:47: DtypeWarning: Columns (8,10,11,14,20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')


✅ Fichier pour le 20250914 téléchargé avec succès.


...

✅ Fichier pour le 20250919 téléchargé avec succès.


C:\Users\jeanc\AppData\Local\Temp\ipykernel_38960\424091.py:47: DtypeWarning: Columns (11,14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')


✅ Fichier pour le 20250920 téléchargé avec succès.
✅ Fichier pour le 20250921 téléchargé avec succès.


C:\Users\jeanc\AppData\Local\Temp\ipykernel_38960\424091.py:47: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zip_ref.open(csv_file_name), sep='\t', header=None, encoding='ISO-8859-1')


✅ Fichier pour le 20250922 téléchargé avec succès.

📦 Consolidation de tous les DataFrames en un seul...

✅ Données chargées dans le DataFrame 'final_df'.
Taille du DataFrame : 1518833 lignes et 59 colonnes.

Aperçu du DataFrame :
            0         1       2     3          4    5    6    7    8    9  \
0  1262227594  20240910  202409  2024  2024.6849  NaN  NaN  NaN  NaN  NaN   
1  1262227595  20240910  202409  2024  2024.6849  NaN  NaN  NaN  NaN  NaN   
2  1262227596  20240910  202409  2024  2024.6849  NaN  NaN  NaN  NaN  NaN   
3  1262227597  20240910  202409  2024  2024.6849  NaN  NaN  NaN  NaN  NaN   
4  1262227598  20240910  202409  2024  2024.6849  NaN  NaN  NaN  NaN  NaN   

   ... 49                       50  51    52         53        54        55  \
0  ...  1            United States  US    US  39.828175  -98.5795        US   
1  ...  4    Xiaomi, Fujian, China  CH  CH07  25.222500  116.0570  10999868   
2  ...  2  Missouri, United States  US  USMO  38.462300  -92.3020    

In [15]:
len(final_df)

817593

In [23]:
final_df[59]

KeyError: 59

In [21]:
all_data

[                 0         1       2     3          4       5              6  \
 0       1262227594  20240910  202409  2024  2024.6849     NaN            NaN   
 1       1262227595  20240910  202409  2024  2024.6849     NaN            NaN   
 2       1262227596  20240910  202409  2024  2024.6849     NaN            NaN   
 3       1262227597  20240910  202409  2024  2024.6849     NaN            NaN   
 4       1262227598  20240910  202409  2024  2024.6849     NaN            NaN   
 ...            ...       ...     ...   ...        ...     ...            ...   
 147347  1262455227  20250910  202509  2025  2025.6849  USAMIL  UNITED STATES   
 147348  1262455228  20250910  202509  2025  2025.6849  USAMIL  UNITED STATES   
 147349  1262455229  20250910  202509  2025  2025.6849     idg     INDIGENOUS   
 147350  1262455230  20250910  202509  2025  2025.6849     iku          INUIT   
 147351  1262455231  20250910  202509  2025  2025.6849     sco       SCOTTISH   
 
           7    8    9  ..